In [18]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("bank-additional-full.csv",sep=";")  # read in the file

In [7]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [8]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [9]:
df.isnull().values.any()  # check nan values

False

In [10]:
df.dtypes  # check types of columns

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

Although there is no nan value in the dataframe, there are many "unknown" strings that are basically the same thing as nan value. It indicates that clients do not mention such information in the phone call. Therefore we come up various techniques to deal with such value.

In [16]:
for col in df.columns:  # loop through columns
    unknown_count = df[col].value_counts().get('unknown')  # find number of rows with unknown for each column
    print(f"{col}: {unknown_count}")

age: None
job: 330
marital: 80
education: 1731
default: 8597
housing: 990
loan: 990
contact: None
month: None
day_of_week: None
duration: None
campaign: None
pdays: None
previous: None
poutcome: None
emp.var.rate: None
cons.price.idx: None
cons.conf.idx: None
euribor3m: None
nr.employed: None
y: None


# NaN Values 

# Technique 1: Removal

As we can see above, the rows of 'unknown' in features martial and jobs, so we are going to just drop them

In [30]:
df.shape

(41188, 21)

In [32]:
df2 = df.copy()

# delete all rows with column 'Age' has value 30 to 40
index_drop = df2[(df2['marital'] == 'unknown') | (df2['job'] == 'unknown') ].index
df2.drop(index_drop, inplace=True)
df2.shape

(40787, 21)

# Technique 2: Replacement

For features housing and loans, we are going to replace them with mode

In [33]:
df2.replace('unknown', np.nan, inplace=True)  # replace it with nan for fillna() usage
df2.isnull().sum()  # check number of nan for each feature

age                  0
job                  0
marital              0
education         1596
default           8436
housing            984
loan               984
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64

In [34]:
df2['housing'].fillna(df2['housing'].mode()[0], inplace=True)  # replacing with mode
df2['loan'].fillna(df2['loan'].mode()[0], inplace=True)

In [35]:
df2.isnull().sum()  # check again

age                  0
job                  0
marital              0
education         1596
default           8436
housing              0
loan                 0
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64

# Technique 3: Classification model

Finally, we are going to use a ML classification model to fill the missing values for the “default” and “education” features. In this case, I choose KNN

In [37]:
#from sklearn.preprocessing import LabelEncoder  
from sklearn.impute import KNNImputer

In [45]:
df2['education'].unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', nan, 'university.degree', 'illiterate'],
      dtype=object)

In [46]:
df2['default'].unique()

array(['no', nan, 'yes'], dtype=object)

In [47]:
# since education and default are both categorical, we have to convert them to numeric type first
ed = df2[["education", "default"]]

ed['education'] = ed['education'].replace(['illiterate', 'basic.4y', 'basic.6y','basic.9y','high.school','professional.course','university.degree'], [0,1,2,3,4,5,6])
ed['default'] = ed['default'].replace(['yes','no'],[1,0])

C:\Users\yukef\AppData\Local\Temp\ipykernel_31404\3278602164.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed['education'] = ed['education'].replace(['illiterate', 'basic.4y', 'basic.6y','basic.9y','high.school','professional.course','university.degree'], [0,1,2,3,4,5,6])
C:\Users\yukef\AppData\Local\Temp\ipykernel_31404\3278602164.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed['default'] = ed['default'].replace(['yes','no'],[1,0])


In [49]:
ed.isnull().sum()

education    1596
default      8436
dtype: int64

In [58]:
imputer = KNNImputer(n_neighbors=5)  # using knn number of neighbours be 5
new_ed = pd.DataFrame(imputer.fit_transform(ed))  # fitting

new_ed.isnull().sum()  # check again

0    0
1    0
dtype: int64

In [61]:
new_ed.rename(columns = {0:'education',1:'default'}, inplace=True)  # change column back to original
# change back to categorical
new_ed['education'] = new_ed['education'].replace([0,1,2,3,4,5,6], ['illiterate', 'basic.4y', 'basic.6y','basic.9y','high.school','professional.course','university.degree'])
new_ed['default'] = new_ed['default'].replace([1,0], ['yes','no'])

In [66]:
df3 = df2.copy()
df3.drop(columns=['education', 'default'], inplace = True)
df3.shape

(40787, 19)

In [67]:
new_ed.shape

(40787, 2)

In [74]:
df5 = pd.concat([df3, new_ed], axis=1,join = 'inner')  # join these two df together
df5

,age,job,marital,housing,loan,contact,month,day_of_week,duration,campaign,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,education,default
0,56,housemaid,married,no,no,telephone,may,mon,261,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,basic.4y,no
1,57,services,married,no,no,telephone,may,mon,149,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,high.school,no
2,37,services,married,yes,no,telephone,may,mon,226,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,high.school,no
3,40,admin.,married,no,no,telephone,may,mon,151,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,basic.6y,no
4,56,services,married,no,yes,telephone,may,mon,307,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,high.school,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40782,33,admin.,married,no,yes,cellular,sep,wed,397,1,...,2,failure,-1.1,94.199,-37.5,0.879,4963.6,yes,professional.course,no
40783,33,technician,married,no,no,cellular,sep,wed,569,1,...,0,nonexistent,-1.1,94.199,-37.5,0.879,4963.6,yes,professional.course,no
40784,32,technician,married,yes,no,cellular,sep,wed,155,1,...,1,success,-1.1,94.199,-37.5,0.879,4963.6,yes,university.degree,no
40785,32,technician,married,no,no,cellular,sep,wed,183,1,...,5,success,-1.1,94.199,-37.5,0.879,4963.6,yes,professional.course,no


In [75]:
df5.shape

(40389, 21)

In [76]:
df5.isnull().sum()  # no NaN anymore

age               0
job               0
marital           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
education         0
default           0
dtype: int64

# Outliers

In [77]:
df5.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,40389.000000,40389.000000,40389.000000,40389.000000,40389.000000,40389.000000,40389.000000,40389.000000,40389.000000,40389.000000
mean,39.973508,257.834732,2.573423,965.855629,0.162619,0.092149,93.564857,-40.443650,3.645441,5169.022380
std,10.353241,259.360195,2.778498,178.374909,0.468528,1.573449,0.572300,4.567413,1.723083,69.692695
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.000000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.354000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.444000,-41.800000,4.857000,5191.000000
75%,47.000000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,6.000000,1.400000,94.465000,-26.900000,5.045000,5228.100000


We can see that for feature like duration, the max is 4918 but the mean is only around 257, which indicates the existence of outliers. We are going to use upper outer fence (upper fence = Q3 + 3*IQR) to deal with it

In [78]:
Q1 = np.percentile(df5['duration'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df['duration'], 75,interpolation = 'midpoint')
IQR = Q3 - Q1

In [80]:
upper_fence = Q3 + 3 * IQR

In [82]:
index_drop2 = df5[(df5['duration'] > upper_fence)].index
df5.drop(index_drop2, inplace=True)  # drop rows that having duration greater than upper fence
df5.shape

(39366, 21)